In [3]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [4]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest


In [5]:

import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif

In [6]:
data = pd.read_csv('dc_combined.csv')


In [7]:

def remove_attack_and_seq_labels(data):
    X =  data.loc[:,~data.columns.str.startswith('Atk')]
    X =  X.loc[:,~X.columns.str.startswith('Attack')]
    X =  X.loc[:,~X.columns.str.startswith('Label_')]
    X = X.drop(['Label', 'Seq'], axis=1)
    return X

In [8]:
X = remove_attack_and_seq_labels(data)


In [9]:
def label_encoder(data, column_name):
  Y = data[column_name]
  label_encoder = LabelEncoder()
  y_encoded = label_encoder.fit_transform(Y)
  return y_encoded

In [10]:
Y = label_encoder(data, 'Label')

In [11]:
def mutual_info(X, Y):
  mutual_info_arr = mutual_info_classif(X, Y)
  series_info = pd.Series(mutual_info_arr)
  series_info.index = X.columns
  series_top = series_info.sort_values(ascending=False)[:20]
  return series_top

In [12]:
result = mutual_info(X, Y)
     

In [13]:
new_data = data[result.keys()]


In [14]:
def min_max_normalisation(X):
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler(feature_range=(-1, 1))
  scaled_data = scaler.fit_transform(X)
  return scaled_data

In [15]:
normalised_data = min_max_normalisation(new_data)


In [16]:
X= normalised_data
y= Y

In [ ]:
# Fit the LOF model
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
y_pred = lof.fit_predict(X)

In [ ]:
# Identify outliers (anomalies)
outliers = np.where(y_pred == -1)[0]

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot of the dataset
plt.figure(figsize=(20, 6))
plt.scatter(X[:, 0], X[:, 1], c='blue', label='Inliers', alpha=0.8)

# Highlight outliers detected by Local Outlier Factor
plt.scatter(X[outliers, 0], X[outliers, 1], c='red', label='Outliers')
plt.title('Outliers Detected by Local Outlier Factor')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()


In [ ]:
# Fit the Isolation Forest model
isolation_forest = IsolationForest(contamination=0.1, random_state=42)
isolation_forest.fit(X)

# Predict anomaly scores
anomaly_scores = isolation_forest.decision_function(X)

# Identify outliers (anomalies)
outliers = np.where(isolation_forest.predict(X) == -1)[0]
print("Indices of outliers:", outliers)

In [ ]:



# Highlight outliers detected by Isolation Forest
plt.scatter(X[outliers, 0], X[outliers, 1], c='red', label='Outliers')
plt.title('Outliers Detected by Isolation Forest')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()